# 科创中心考试答题卡

答卷要求
---
1.作答形式：以小组形式（3-4人)进行答题。可以随意在互联网寻找相关资料，不可小组之间相互抄袭或找外援。（被发现破坏公平行为直接取消全组成绩作废）前三题为技术题，需要代码实现。最后一题为简答题，也可以进行实现论证，会根据作答情况进行酌情加分。


2.提交要求：尽量以ipython文件格式（本文件）于2022年1月10日发送至邮箱2286988225@qq.com（邮件名：科创中心考试答题+小组名（自拟）/个人姓名），完成度在60%以上的团队将对结果进行展示交流。


3.卷面要求：将代码在所备注的部分进行填写。可以增加行数，但不要填到别的题的答题范围内。尽量加上注释，卷面可读性会提高小组评分。（以让阅卷人尽量能清晰明了的看懂代码为目的）。


3.在填写名字的地方备注上分工如下例子：

信息填写
---
例：答卷人1姓名：杨雨薇（第一题数据分析，第三题可视化）


答卷人1学号：2020****


答卷人2姓名：


答卷人2学号：


答卷人3姓名：


答卷人3学号：

数据说明如下:
---
数据为x省x市x区x年x月x日-xx年x月x日的处罚事故数据。数据包括报警时间、报警地点等各类警情信息。某大队要根据所给出的数据，对数据进行分析，并根据该数据进行合理的路径规划（也可以根据历史数据进行预测未来一周的路径）。

第一题：
---
根据所给出的事故双方的当事人的身份证号，判断其身份证号是否合法，并对身份证号、电话号码进行解析（ id_validator，phonenumber）获取其年龄、户籍、目前通讯地址（电话号码所在地）。通过统计身份证号，以及所给人员的信息，对事故多发人员的户籍地、年龄、居住社区，并进行可视化，对高发事故人员画像。

第二题：
---
根据标注地点从地图API获取其具体坐标

In [58]:
# 第二题题目：
# 根据标注地点从地图API获取其具体坐标

import urllib
import json
from requests.api import request
from pandas import Series, DataFrame
import pandas as pd

##################################################################################
# 函数功能：调取高德地图API获得信息
# 作者：胡文强
# 主要思路：通过获取的数据，构造URL并通过requests库向高德地图 API 访问得到返回值，再进行数据处理得到有效信息
# 具体实现：
# 1. 获取值：
#   1. returned_information_format: 返回信息的类型：json 还是 xml，我觉得json更分别分析
#   2. input_x, input_y: 输入的 GPS 的 X、Y 坐标
#   3. input_key: 用户 Key
#   4. result_radius: 查询的半径
#   5. returned_information_kind: 是返回所有的结果（all）还是单个（base）
# 2. 通过 urllib 的 request 来调取API得到返回值，如果正常则返回 string 类型的 returned_result，如果有异常则返回 string 类型的 '-1'
# 3. 将返回值给别的函数（get_processed_location）处理，得到所需数据
# 备注：通过孟浩阳的高德地图账号申请到 key 为：aad49afa17b46e85e060bbe252f25a80
def get_location(returned_information_format, input_x, input_y, input_key, returned_information_kind):
    # Url example: https://restapi.amap.com/v3/geocode/regeo?output=xml&location=116.310003,39.991957&key=<用户的key>&radius=1000&extensions=all (all/base)
    url = 'https://restapi.amap.com/v3/geocode/regeo?output=' + str(returned_information_format).strip() + 'xml&location=' + str(input_x).strip() + ',' + str(input_y).strip() + '&key=' + str(input_key).strip() + '&radius=1000' + '&extensions=' + str(returned_information_kind).strip()
    try:
        returned_result = urllib.request.urlopen(url).read()
        returned_result = returned_result.decode()
        returned_result = str(returned_result)
        return returned_result
    except:
        return '-1'
##################################################################################

##################################################################################
# 函数功能：处理通过地图API获取到的json信息得到地址输出
# 作者：胡文强
# 主要思路：先进行校验，若收录则对信息进行处理，若无法得到任何结果则返回 “未收录”
# 具体实现：
# 1. 进行校验，若 return_result 不为 -1 （即网络正常，可访问）则开始处理：
#   1. 将 returned_result 转为json数据
#   2. 从中提取出 regeocode 中的 formatted_address 项，如果失败返回 “error: 坐标有误！”
# 2. 若为 -1 则返回结果 “error: 网络错误！”
def get_processed_location(returned_result):
    if (returned_result != '-1'):
        try:
            temp = json.loads(returned_result)
            processed_location = temp["regeocode"]["formatted_address"]
            return processed_location
        except:
            return 'error: 坐标有误!'
    else:
        return 'error: 网络错误!'
##################################################################################

##################################################################################
# 函数功能：读取表格中的数据并处理，完成后进行输出
# 作者：孟浩阳，胡文强
# 主要思路：读入表格中的GPS列信息，利用上述函数进行处理，之后输出到Excel中
# 具体实现：如代码中注释所述
# Bugs：有异常的无法显示异常信息，只会显示为空
if __name__ == "__main__":
    # 引入考试数据
    df = pd.read_excel("./考试数据 (原件).xls") # 被胡文强改为了相对路径
    # 对GPS两列进行数据提取
    GPS_get_x = df['GPS_X'].to_string(header=False, index=False).split('\n')
    GPS_get_x = GPS_get_x[1:]

    GPS_get_y = df['GPS_Y'].to_string(header=False, index=False).split('\n')
    GPS_get_y = GPS_get_y[1:]
    # AGEADDRESS_list = pd.DataFrame()#建造一个空列表
    output_list = pd.DataFrame()
    # 将身份证提取并作年龄处理
    for GPS_x, GPS_y in zip(GPS_get_x, GPS_get_y):
        GPS = get_processed_location(get_location('json', GPS_x, GPS_y, 'aad49afa17b46e85e060bbe252f25a80', 'base'))
        output_list = output_list.append(pd.DataFrame({'GPS地址': [GPS]}), ignore_index=True)

    # 将年龄的字符串放入空列表中，并创建数组写入excel
    output_list.to_excel(r'./GPS数据结果.xls', index = False, header=True)
##################################################################################

第三题：
---
根据第二题所获取的地理坐标信息，对2020年12月1日的事故处理路线，基于案件的警情以及实际坐标进行合理的路径规划（最好能用可视化方式展现合理的规划方案）。（假设处理每起事故需5分钟）

In [59]:
###第三题答案代码

第四题：
---
根据以上所给数据给出你们觉得可行的数据分析方向，方法，以及可行的技术路线，理论依据。（如果实现可加分）

第四题答题区域：